In [60]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
from dataclasses import dataclass

In [25]:
act = pd.read_excel('act_21.xlsx')
ost = pd.read_excel('ost_26.07.xlsx')

In [24]:
indexes_to_drop = ost[ost['Номенклатура'].str.contains('ДАЛ', na=False)].index
ost_drop = ost.drop(indexes_to_drop, axis=0)
drop = len(ost) - len(ost_drop)
drop


29

In [63]:
date_test = '16.01.2023'
format_date = '%d.%m.%Y'
date = datetime.strptime(date_test, format_date).date()
today = datetime.today().date()
offset = today - date
if offset > timedelta(days=180):
    print('tr')


tr


In [125]:
@dataclass
class Beer:
    name: str
    alc_prod: str
    count: str
    reference: str
    new_references: list

In [126]:
def name_reference_check(beer, act, ost):
    ost_name = ost[ost['Номенклатура'].str.contains(beer.name, na=False)]
    for index, row in ost_name.iterrows():
        if row['Количество'] >= beer.count and beer.count != 0:
            beer.new_references.append([beer.count, row['Справка 2']])
            row['Количество'] -= beer.count
            beer.count = 0
            ost.loc[ost['N'] == row['N'], 'Количество'] = row['Количество']
        if row['Количество'] < beer.count and row['Количество'] != 0:
            beer.new_references.append([row['Количество'], row['Справка 2']])
            beer.count -= row['Количество']
            row['Количество'] = 0
            ost.loc[ost['N'] == row['N'], 'Количество'] = row['Количество']
    return ost

In [127]:
def alc_prod_reference_check(beer, act, ost):
    ost_name = ost[ost['Алкогольная продукция'].str.contains(beer.alc_prod, na=False)]
    for index, row in ost_name.iterrows():
        if row['Количество'] >= beer.count and beer.count != 0:
            beer.new_references.append([beer.count, row['Справка 2']])
            row['Количество'] -= beer.count
            beer.count = 0
            ost.loc[ost['N'] == row['N'], 'Количество'] = row['Количество']
        if row['Количество'] < beer.count and row['Количество'] != 0:
            beer.new_references.append([row['Количество'], row['Справка 2']])
            beer.count -= row['Количество']
            row['Количество'] = 0
            ost.loc[ost['N'] == row['N'], 'Количество'] = row['Количество']
    return ost

In [136]:
def creating_correct_rows(row, beer, act, act_del):
    #while beer.new_references != []:
    for ref in beer.new_references:
        row['Количество'] = ref[0]
        row['Справка 2'] = ref[1]
        #act = pd.concat([act, row])
        act.loc[len(act)] = row
    if beer.count != 0:
        row['Количество'] = beer.count
        row['Справка 2'] = 'NaN'
        act_del.loc[len(act)] = row
    return act, act_del


In [138]:
act_new = act.drop(act.index, axis=0)
act_del = act.drop(act.index, axis=0)
for index, row in act.iterrows():
    beer = Beer(name=row['Номенклатура'], alc_prod=row['Алкогольная продукция'],
                count=row['Количество'], reference=row['Справка 2'], new_references=[])
    if ost['Номенклатура'].isin([beer.name]).any():
        ost = name_reference_check(beer, act, ost)
        print(beer)
    if ost['Алкогольная продукция'].isin([beer.alc_prod]).any() and beer.count != 0:
        ost = alc_prod_reference_check(beer, act, ost)
        print(beer)
    act.loc[index, 'Справка 2'] = 'del'
    act_new, act_del = creating_correct_rows(row, beer, act_new, act_del)
    

Beer(name='Реворт Лава Ран 0,5', alc_prod='Пивной напиток непастеризованный нефильтрованный неосветленный светлый ГОЗЕ 5 ЛАВА РАН', count=0, reference='FB-000005935005873', new_references=[[1, 'FB-000005935005873']])
Beer(name='Сигал Олег Салти Бомб 0,5', alc_prod='Пиво светлое непастеризованное нефильтрованное неосветленное "Саур 1"', count=0, reference='FB-000005952888398', new_references=[[1, 'FB-000005952888397']])
Beer(name='Бирвария Дон Симон 0,33', alc_prod='Пиво темное непастеризованное нефильтрованное неосветленное "Дон Симон"', count=0, reference='FB-000005670604036', new_references=[[4, 'FB-000005670604036']])
Beer(name='Ред Баттон Янтарное 1 л', alc_prod='Пиво светлое непастеризованное нефильтрованное неосветленное "Бо', count=0, reference=nan, new_references=[[3, 'FB-000006004784800']])
Beer(name='Аляска Бутлегер Олд Фэшн 0.5', alc_prod='Пиво светлое непастеризованное нефильтрованное неосветленное "Св', count=0, reference='FB-000006001370558', new_references=[[2, 'FB-00000

C:\Users\nevok\AppData\Local\Temp\ipykernel_724\722907173.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ost_name = ost[ost['Алкогольная продукция'].str.contains(beer.alc_prod, na=False)]


In [139]:
act_del

,N,Индекс акцизной марки,Номенклатура,Упаковка,Ед. изм.,Алкогольная продукция,Количество,Цена,Сумма,Справка 2,Дата подтверждения ЕГАИС
8,9,NaN,Бирвария Жижка от салата 0.5,NaN,шт,Пивной напиток нефильтрованный непастеризованн...,1,330,330,NaN,18.04.2023
22,22,NaN,Митра Локал 90 0.5,NaN,шт,Пивной напиток непастеризованный нефильтрованн...,2,320,512,NaN,NaN


In [130]:
act_new['N'] = act_new.apply(lambda row: row.name + 1, axis=1)
act_new.to_excel('act_ref.xlsx', index=False)
act_del.to_excel('act_del.xlsx', index=False)